In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("amazon_reviews_multi", "es")
dataset

Reusing dataset amazon_reviews_multi (/home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [3]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(20_000))

Loading cached shuffled indices for dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-e32eb148007d7067.arrow


In [4]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "From: https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"])

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0727973,product_es_0688938,reviewer_es_0468676,1,SOIS UNOS HDP me mandasteis un produccto que no era el que estabais vendiendo. Un p8 lite me mandais en vez del que vendeis.,ESTAFA,es,wireless
1,es_0806549,product_es_0547116,reviewer_es_0470608,3,Muy basico y calidad media,Carraca,es,home_improvement
2,es_0141795,product_es_0142033,reviewer_es_0644301,5,Se ajusta al producto descrito. Muy buena relación caludad-precio,Buen producto,es,toy
3,es_0051189,product_es_0865715,reviewer_es_0253018,4,"De momento bien, espero que me dure",Buen montaje,es,home_improvement
4,es_0042184,product_es_0643491,reviewer_es_0978177,4,Acorde con las especificaciones,Buen precio / calidad,es,wireless
5,es_0403162,product_es_0441084,reviewer_es_0939490,4,"El kit es muy completo, el soporte del fabricante es sobresaliente. Muy fácil de instalar por lo completa que viene la pantalla, sus accesorios y sus instrucciones.",Kit completo y de fácil instalación,es,wireless
6,es_0136866,product_es_0137431,reviewer_es_0487268,3,Un corre pasillos muy económico pero con poca estabilidad si el niño/niña no sabe andar aún. Las ruedas traseras deberían estar más abiertas o ser más gruesas,Bien pero mejorable,es,toy
7,es_0360088,product_es_0986734,reviewer_es_0234926,4,"Es un cortador de piña que la verdad es muy práctico y fácil de usar, tan sólo dar unas vueltas hasta llegar hasta la parte de abajo de la piña y tirar hacia arriba y ya salen todas las rodajas listas para comer.El cortador viene un poco sucio pero sólo es polvo se lava y ya está.",Juanma,es,home
8,es_0123657,product_es_0016289,reviewer_es_0734531,3,Todo lo que llevo afilado hasta el momento me lo ha dejado de maravilla.,Esta bastante bien,es,home
9,es_0765509,product_es_0752738,reviewer_es_0806997,2,"La devolvi porque mi hija de nueve meses no cabia, literalmente, en el asiento, las medidas en si son más justitas que otras tronas, nosotros ibamos buscando una que nos durara un tiempo.",pequeña,es,baby_product


In [6]:
dataset.set_format("pandas")

In [7]:
df = dataset["train"][:]

In [8]:
df.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0614019,product_es_0489972,reviewer_es_0452711,3,La montarlo se rompió una rueda debido a mater...,.,es,furniture
1,es_0106878,product_es_0192285,reviewer_es_0332773,2,"El servicio ha sido muy bueno, me ha llegado 2...",Primeras impresiones,es,pc
2,es_0894107,product_es_0849920,reviewer_es_0811760,5,Funciona genial y la llevo conmigo en todas la...,.,es,camera
3,es_0611608,product_es_0261733,reviewer_es_0873700,4,"Me ha encantado el olor, pero se va a la media...",Olor excelente,es,beauty
4,es_0374593,product_es_0528114,reviewer_es_0342501,5,Muy divertida y se ajusta a la descripción. En...,Envío rápido,es,toy


In [9]:
df["stars"].value_counts()

5    4065
4    4045
3    4032
1    3960
2    3898
Name: stars, dtype: int64

## Tokenize

In [10]:
from transformers import AutoTokenizer

# model_checkpoint = "bertin-project/bertin-roberta-base-spanish"
model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
tokenizer.vocab_size

50262

In [12]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [13]:
encoded_str = tokenizer.encode("¡hola, me llamo lewis!")

for token in encoded_str:
    print(token, tokenizer.decode([token]))

0 <s>
1465 ¡
12616 hola
66 ,
503  me
17111  llamo
532  le
19514 wis
55 !
2 </s>


In [14]:
import transformers
transformers.__version__

'4.9.2'

In [15]:
def tokenize_text(examples):
    return tokenizer(examples["review_body"], truncation=True)

In [16]:
dataset.reset_format()

In [17]:
# dset = dataset["train"].train_test_split()

In [18]:
encoded_dataset = dataset.map(tokenize_text, batched=True)
encoded_dataset

Loading cached processed dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-5891b043fe5d9345.arrow
Loading cached processed dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-82c7b30bde2d5822.arrow
Loading cached processed dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-5b22c1fadf14e8e7.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'language', 'product_category', 'product_id', 'review_body', 'review_id', 'review_title', 'reviewer_id', 'stars'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'language', 'product_category', 'product_id', 'review_body', 'review_id', 'review_title', 'reviewer_id', 'stars'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'language', 'product_category', 'product_id', 'review_body', 'review_id', 'review_title', 'reviewer_id', 'stars'],
        num_rows: 5000
    })
})

In [19]:
def binarize_labels(examples):
    if examples["stars"] > 3:
        label = 1
    else:
        label = 0
    return {"labels": label}

In [20]:
encoded_dataset = encoded_dataset.map(binarize_labels)

Loading cached processed dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-7443381e9b27829a.arrow
Loading cached processed dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-3dac11b4268fb4d2.arrow
Loading cached processed dataset at /home/lewis/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-ba621aa6467e4a29.arrow


In [21]:
# label_mapping = {idx+1: idx for idx in range(5)}
# label_mapping

# encoded_dataset = encoded_dataset.map(lambda x: {"labels": label_mapping[x["stars"]]})

In [22]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at BSC-TeMU/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense

In [23]:
# encoded_dataset.set_format("pt")

In [24]:
# model(**encoded_dataset["train"][:1])

In [25]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted") 
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [26]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(encoded_dataset["train"]) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch", 
                                 logging_steps=logging_steps)

In [27]:
from transformers import Trainer

trainer = Trainer(model=model, 
                  args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=encoded_dataset["train"],
                  eval_dataset=encoded_dataset["validation"],
                  tokenizer=tokenizer)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: language, product_category, reviewer_id, review_title, review_id, review_body, product_id, stars.
***** Running training *****
  Num examples = 20000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3125
/home/lewis/miniconda3/envs/hf/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.286933,0.882400,0.882042
2,0.266500,0.351466,0.882400,0.881722
3,0.266500,0.433992,0.873800,0.874459
4,0.079500,0.581961,0.877000,0.876932
5,0.079500,0.677518,0.872400,0.872072


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: language, product_category, reviewer_id, review_title, review_id, review_body, product_id, stars.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 32
Saving model checkpoint to results/checkpoint-625
Configuration saved in results/checkpoint-625/config.json
Model weights saved in results/checkpoint-625/pytorch_model.bin
tokenizer config file saved in results/checkpoint-625/tokenizer_config.json
Special tokens file saved in results/checkpoint-625/special_tokens_map.json
/home/lewis/miniconda3/envs/hf/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corres

TrainOutput(global_step=3125, training_loss=0.1422505355834961, metrics={'train_runtime': 645.8102, 'train_samples_per_second': 154.844, 'train_steps_per_second': 4.839, 'total_flos': 6779138563555200.0, 'train_loss': 0.1422505355834961, 'epoch': 5.0})

In [29]:
trainer.evaluate(encoded_dataset["test"])

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: language, product_category, reviewer_id, review_title, review_id, review_body, product_id, stars.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 32
/home/lewis/miniconda3/envs/hf/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.2728419899940491,
 'eval_accuracy': 0.8878,
 'eval_f1': 0.8876700706373296,
 'eval_runtime': 9.0253,
 'eval_samples_per_second': 553.996,
 'eval_steps_per_second': 17.395,
 'epoch': 5.0}